### Preprocessing  
Sofia

In [7]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# list of file name strings from S1 and S2
S1_files = sorted(glob.glob('../S1_Dataset/d1p*'))
S2_files = sorted(glob.glob('../S2_Dataset/d2p*'))

# TODO: when done with program, merge all data from S1 and S2 into 1 giant dataset
# for testing only using 1 file for now
file = S1_files[0]

df = pd.read_csv(file, header=None)
df.columns = [
    'time',
    'frontal accel',
    'vertical accel',
    'lateral accel',
    'antenna id',
    'rssi',
    'phase',
    'frequency',
    'activity',
]
activities = {
    1: 'sit on bed',
    2: 'sit on chair',
    3: 'lying',
    4: 'ambulating',
}
df.replace({'activity': activities}, inplace=True)

# TODO: add 3 columns activity status, frequency of activity type (time over total time), and gender 
# our target is "activity status" (inactive or active) with our independent variables being activity type, time, rssi, frontal accel, vertical accel, lateral accel, gender, frequency (of activity type)


# TODO: preprocess data and split into test and training sets using df
# normalize data 
# split 25:75 training and test

### Building + Testing Model

**Hyper param testing**  
Martin  


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor

# TODO: add hyperparameters that we will be testing as parameters for this function (Martin)
def create_model(): 
    print("NOT IMPLEMENTED")
    # TODO: implement our nueral network structure using SGD as the optimization technique (Martin)

    # maybe use tf.keras.sequential here?
    model = tf.keras.sequential()
    return model
    
# TODO: add hyper parameters that we will be testing to param_grid (Martin)
param_grid = {
    "activation_function_hidden": [],
    "activation_function_output": [],
    "hidden_units": [],
    "hidden_layers": [],
    "regularizer": [],
    "reg_param": [],
    "momentum": [],
    "batch_size": [],
    "epochs": [],
    "learning_rate": [],
}

keras_model = KerasRegressor(build_fn=create_model, verbose=0)

# TODO: for GridSearchCV modify any other params that we need (scoring, cv, num_jobs, etc) (Martin)
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid)
grid_result = grid.fit(X_train_scaled, y_train)

# TODO: implement model evaluation and print/graph anything needed for visualization